In [1]:
from faster_config import Config
from faster_data_source import DataSource
from faster_file_manager import FileManager
from faster_metrics import MetricsAggregator
from faster_model_evaluator import ModelEvaluator
from faster_model_trainer import ModelTrainer
from faster_plotter import Plotter
from faster_model_loader import ModelLoader

In [2]:
Config.set_local_settings()
file_manager = FileManager()
data_source = DataSource(file_manager)
model_loader = ModelLoader(file_manager)
model_trainer = ModelTrainer(file_manager)
metrics_aggregator = MetricsAggregator(file_manager)
plotter = Plotter(file_manager, metrics_aggregator)

 Loading Annotations:  InBreast-COCO\train\_annotations.coco.json 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
model = model_loader.new_pretrained_model()

data_source.start_kfold()
datasets, dataloaders = data_source.testing()
train_dataset, valid_dataset = datasets
train_loader, valid_loader = dataloaders


 New Model:  Faster-RCNN 
 Loading Annotations:  InBreast-COCO\test\_annotations.coco.json 
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
batch_train = next(iter(train_loader))
batch_valid = next(iter(valid_loader))

In [5]:
model_trainer.fit(model, train_loader, valid_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory ./lightning_logs\InBreast-COCO\Faster-RCNN\test\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name        | Type       | Params
-------------------------------------------
0 | faster_rcnn | FasterRCNN | 41.3 M
-------------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 6. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.w

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [4]:
# Validation - Threshold Optimization

best_model = model_loader.load_best_model()
model_evaluator = ModelEvaluator(best_model, plotter)

threshold = 0.001
valid_metrics = model_evaluator.evaluate(valid_dataset, valid_loader, threshold, save_plots=True)
valid_metrics

 Loading Model:  lightning_logs\InBreast-COCO\Faster-RCNN\test\checkpoints\epoch=0-step=1.ckpt 
 Evaluating on threshold:  0.001 


{'Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.0,
 'Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ]': 0.0,
 'Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ]': 0.0}